## Imports

In [7]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torchaudio.transforms import MelSpectrogram


## Loading Data

- `train_audio/`: short recordings of individual species with labels - *supervised learning*
- `train_soundscapes/`: long recordings without labels - *unsupervised learning*

In [6]:
from pathlib import Path

data_dir = Path('/data')
train_dir = data_dir / 'train_audio'

train_dir.exists()

False